In [1]:
# Import packages
import os
import pathlib
import zipfile

# import relative path replace / with . subdirectory.load_plot_mode
import geopandas as gpd
import load_model
import matplotlib.pyplot as plt
from riverrem.REMMaker import REMMaker, clear_osm_cache
import requests
import rioxarray as rxr
from rioxarray.merge import merge_arrays

/Users/lchipman/earth-analytics/data/watershed-project is now the working directory


In [2]:
# Set working directory
working_dir = os.path.join(
    pathlib.Path.home(), 'earth-analytics', 'data', 'watershed-project')

# Try/Except Block   
try:
    os.chdir(working_dir)
except:
    print('{} does not exist. Creating...'.format(working_dir))
    os.makedirs(working_dir)
    os.chdir(working_dir)
else:
    print('{} is now the working directory'.format(working_dir))

/Users/lchipman/earth-analytics/data/watershed-project is now the working directory


# Load the site information into dictionaries

## Load the LiDAR DTMs
* Download and unzip lidar data

ideally would download from CO hs site, but there is no direct link. I uploaded zip files to git, and had to change the file content names bc otherwise  running into issues unzipping - the extracted files include root directory in filename, and the name is not callable

In [3]:
# Call function to get urls for lidar zipfiles, add to dictionary
site_names = ['applevalley', 'hallmeadows', 'hallmeadows2', 'highway93']
site_lidar_urls = load_model.get_lidar_url(site_names=site_names)

In [4]:
# Download and unzip lidar data, load and save lidar DTMs to dictionaries
for site in site_lidar_urls:
    site['lidar_dtm'] = (load_model.load_dtm(data_url=site['lidar_url'], 
                                     site_name=site['site_name'],
                                     file_name=site['zip_filename']))
# Merge the 2 hallmeadows tiles and replace dtm:
for site in site_lidar_urls:
    if site['site_name'] == 'hallmeadows':
        site['lidar_dtm'] = (merge_arrays(dataarrays = [site_lidar_urls[1]['lidar_dtm'], 
                                                        site_lidar_urls[2]['lidar_dtm']]))
    if site['site_name'] == 'hallmeadows2':
        site_lidar_urls.remove(site)

## Download the area of interest (AOI)

In [6]:
# Function to get the bounding polygon and save as gdf
# Want to move this to load_model function and delete here, but when I  
# run load_model.get_boundary_gdf(), the open gdf step doesn't work, might be directory issue?
def get_boundary_gdf(data_url, site_name):
    """Downloads boundary shapefiles and open as a gdf
    
    Parameters
    ------------
    data_url: str
        Url for the boundary shapefiles (zipfile)
    
    site_name: str
        The site name.
        
    Returns
    ------------
    gdf: geodataframe
        A geodataframe containing the boundary geometry.
    """
    override_cache = False
    data_path = os.path.join('shapefiles.zip')
    
    # Cache data file
    if (not os.path.exists(data_path)) or override_cache:
        print('{} does not exist. Downloading...'.format(data_path))
        # Download full data file as zipfile
        response = requests.get(data_url)

        # Write in respose content using context manager
        with open(data_path, 'wb') as data_file:
            data_file.write(response.content)
            
    with zipfile.ZipFile(data_path, 'r') as shape_zipfile:
        shape_zipfile.extractall(working_dir)
        
    data_path=os.path.join('shapefiles',
                           '{}_bounding_polygon'.format(site_name),
                           'Bounding_Polygon.shp')
    print(data_path)

    # Open the bounding polygon as gdf
    try:
        gdf = gpd.read_file(data_path)
        return gdf
    except:
        print('There is no bounding polygon for the {} site, ' 
              'skipping this site'.format(site_name))

In [7]:
# Call function to create bounding polygon gdf for each site, save to site_lidar_urls
# why isn't this working in the .py file? load_model.get_boundary_gdf()
shape_url = ('https://github.com/lechipman/watershed-project/'
                 'releases/download/v2.0.0/shapefiles.zip')

for site in site_lidar_urls:
    site['bounding_polygon'] = (get_boundary_gdf(
                                data_url=shape_url, 
                                site_name=site['site_name']))

shapefiles/applevalley_bounding_polygon/Bounding_Polygon.shp
shapefiles/hallmeadows_bounding_polygon/Bounding_Polygon.shp
shapefiles/highway93_bounding_polygon/Bounding_Polygon.shp


## Load the UAV DTMs and REMs

In [ ]:
# Call function to download the UAV DTMs and REMs and add to dictionary
# Note at this step, the main dictionary name is changes to site_data_dictionary
# Because it now contains UAV and LiDAR data
site_data_dictionary = load_model.get_uav_dtms(site_data_dictionary=site_lidar_urls)

## Clip the Lidar and UAV DTMs to AOI

In [ ]:
# Clip the lidar dtm to bounding polygon, convert to meters, and add to dictionary
for site in site_data_dictionary:
    site['lidar_clipped_dtm'] = (load_model.dtm_clip(site_name = site['site_name'],
                                          site_dtm = site['lidar_dtm'],
                                          clip_gdf = site['bounding_polygon'],
                                          is_lidar=True))*0.3048
# Clip the uav dtm to bounding polygon and add to dictionary
for site in site_data_dictionary:
    site['uav_clipped_dtm'] = (load_model.dtm_clip(site_name = site['site_name'],
                                        site_dtm = site['uav_dtm'],
                                        clip_gdf = site['bounding_polygon'],
                                        is_lidar=False))

## Plot the LiDAR and UAV DTMs

In [ ]:
# Call function to plot UAV and Lidar DTMs
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plot_title = ['Apple Valley UAV DTM', 
              'Hall Meadows UAV DTM',
              'Highway93 UAV DTM']
for i,axe in enumerate(axes.flatten()):
    load_model.plot_model(model=site_data_dictionary[i]['uav_clipped_dtm'], 
                   title=plot_title[i],
                   coarsen=True,
                   xpix=20,
                   ypix=20,
                   ax=axe)
    
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plot_title = ['Apple Valley LiDAR DTM', 
              'Hall Meadows LiDAR DTM',
              'Highway93 LiDAR DTM']
for i,axe in enumerate(axes.flatten()):
    load_model.plot_model(model=site_data_dictionary[i]['lidar_clipped_dtm'], 
                   title=plot_title[i],
                   coarsen=False,
                   #xpix=20,
                   #ypix=20,
                   ax=axe)

## To do for steps above

update plot_model to add colorbar and update binning

## Plot the LiDAR and UAV DTM histograms

In [ ]:
# Call function to plot histogram of lidar and UAV DTMs
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plot_title = ['Apple Valley',
              'Hall Meadows',
              'Highway93']

for i, axe in enumerate(axes.flatten()):
    load_model.plot_hists(model=site_data_dictionary[i]['uav_clipped_dtm'], 
               titles=plot_title[i],
               main_title='UAV DTM Histograms',
               color=('cyan'),
               ax=axe, 
               fig=fig)
    
# Call function to plot histogram of lidar DTMs
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plot_title = ['Apple Valley',
              'Hall Meadows',
              'Highway93']

for i, axe in enumerate(axes.flatten()):
    load_model.plot_hists(model=site_data_dictionary[i]['lidar_clipped_dtm'], 
               titles=plot_title[i],
               main_title='LiDAR DTM Histograms',
               color=('cyan'),
               ax=axe,
               fig=fig)

## Create LiDAR REMs Using REMMaker Tool

In [ ]:
# Create LiDAR REMs for all sites with run_rem_maker function, add to dict
for site in site_data_dictionary:
    load_model.run_rem_maker_lidar(site_name=site['site_name'])
    lidar_rem_path = os.path.join(site['site_name'],
                                  'remmaker_lidar',
                                  '{}_lidar_dtm_REM.tif').format(site['site_name'])
    site['lidar_remmaker'] = rxr.open_rasterio(lidar_rem_path, masked=True)

## Create UAV REMs Using REMMaker Tool

In [ ]:
for site in site_data_dictionary:
    load_model.run_rem_maker(site_name=site['site_name'], k=100)
    uav_rem_path = os.path.join(site['site_name'],
                                  'remmaker',
                                  '{}_dtm_REM.tif').format(site['site_name'])
    site['uav_remmaker'] = rxr.open_rasterio(uav_rem_path, masked=True)

## Plot the UAV and LiDAR Derived REMs

In [ ]:
# Call function to plot UAV REMs
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plot_title = ['Apple Valley-UAV REMMaker', 
              'Hall Meadows-UAV REMMaker',
              'Highway93-UAV REMMaker']
for i,axe in enumerate(axes.flatten()):
    load_model.plot_model(model=site_data_dictionary[i]['uav_remmaker'], 
                   title=plot_title[i],
                   coarsen=True,
                   xpix=100,
                   ypix=100,
                   ax=axe)
# Call function to plot lidar REMs
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plot_title = ['Apple Valley-LiDAR REMMaker', 
              'Hall Meadows-LiDAR REMMaker',
              'Highway93-LiDAR REMMaker']
for i,axe in enumerate(axes.flatten()):
    load_model.plot_model(model=site_data_dictionary[i]['lidar_remmaker'], 
                   title=plot_title[i],
                   coarsen=False,
                   #xpix=100,
                   #ypix=100,
                   ax=axe)

## LiDAR and UAV REMMaker Histograms

In [ ]:
# Call function to plot histogram of UAV REMs
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plot_title = ['Apple Valley',
              'Hall Meadows',
              'Highway93']

for i, axe in enumerate(axes.flatten()):
    load_model.plot_hists(model=site_data_dictionary[i]['uav_remmaker'], 
               titles=plot_title[i],
               main_title='UAV REMMaker Histograms',
               color=('cyan'),
               fig=fig,
               ax=axe)
# Call function to plot histogram of lidar REMs
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plot_title = ['Apple Valley',
              'Hall Meadows',
              'Highway93']

for i, axe in enumerate(axes.flatten()):
    load_model.plot_hists(model=site_data_dictionary[i]['lidar_remmaker'], 
               titles=plot_title[i],
               main_title='LiDAR REMMaker Histograms',
               color=('cyan'),
               fig=fig,
               ax=axe)

## Flood mapping
* set a threshold value that represents flooding level (e.g., anything less than 3ft will be innundated)
* for both the uav and lidar rems, calculate the sum of the #points that are <threshold, and multiply by pixel size - this represents the area flooded
* can loop through several flood levels and compare uav/lidar with a scatter plot of flooded area

In [ ]:
# Calculate difference in REMs - need to match resolution
#for site in site_data_dictionary:
#    site['dod_array'] = site['uav_remmaker'] - site['lidar_remmaker']
site_data_dictionary[0]['uav_remmaker'].squeeze()

In [ ]:
# .squeeze()?
threshhold_das = []
masked_da = site_data_dictionary[0]['uav_remmaker'].where(site_data_dictionary[0]['uav_remmaker'] < 1)
masked_da.plot()

In [ ]:
masked_da.sum()
# plot contours, robust=True, cmap
# 1. Find streamflow for return period using the discharge record
# 2. Use the field measurements/rating curve to find the corresponding gage height
# 3. Find the gage height for when the LiDAR was taken
# 4. Subtract the “normal” gage height from the flood gage height
# 5. Mask the REM at the gage height difference


#or number of pixels at different intervals vs gage height at lidar date
# count pixels = sum of raster

In [ ]:
masked_da_2 = site_data_dictionary[0]['uav_remmaker'].where(site_data_dictionary[0]['uav_remmaker'] < .5)
masked_da_2.sum()